Girish Joshi 20BCT0132

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Unizipping Dataset


In [8]:
!unzip '/content/drive/MyDrive/archive.zip'

Archive:  /content/drive/MyDrive/archive.zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: test_da

##Data Augmentation

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen =  ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range = 0.2)
test_gen =  ImageDataGenerator(rescale=1./255) 

In [9]:
train = train_gen.flow_from_directory('/content/train_data/train_data',
                                      target_size=(224,224),
                                      class_mode = 'categorical',
                                      batch_size=8)

test = train_gen.flow_from_directory('/content/test_data/test_data',
                                      target_size=(224,224),
                                      class_mode = 'categorical',
                                      batch_size=8)

Found 150 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


##CNN

In [10]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

In [11]:
model = Sequential()
model.add(Convolution2D(30,(3,3),20,activation = 'relu',input_shape =(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(80,activation='relu'))
model.add(Dense(16,activation='softmax'))

In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(train,batch_size=8,validation_data = test,epochs=10)

Epoch 1/10
19/19 [==============================] - 103s 5s/step - loss: 2.7742 - accuracy: 0.0667 - val_loss: 2.6784 - val_accuracy: 0.1274
Epoch 2/10
19/19 [==============================] - 121s 7s/step - loss: 2.6383 - accuracy: 0.1600 - val_loss: 2.6497 - val_accuracy: 0.1783
Epoch 3/10
19/19 [==============================] - 92s 5s/step - loss: 2.5220 - accuracy: 0.1933 - val_loss: 2.6318 - val_accuracy: 0.1656
Epoch 4/10
19/19 [==============================] - 89s 5s/step - loss: 2.4537 - accuracy: 0.2333 - val_loss: 2.5977 - val_accuracy: 0.2293
Epoch 5/10
19/19 [==============================] - 89s 5s/step - loss: 2.3379 - accuracy: 0.2867 - val_loss: 2.5773 - val_accuracy: 0.2357
Epoch 6/10
19/19 [==============================] - 90s 5s/step - loss: 2.2885 - accuracy: 0.2933 - val_loss: 2.5704 - val_accuracy: 0.2102
Epoch 7/10
19/19 [==============================] - 90s 5s/step - loss: 2.1850 - accuracy: 0.3133 - val_loss: 2.5358 - val_accuracy: 0.1911
Epoch 8/10
19/19 [

## VGG16 Model - For Better Acccuracy

In [14]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [15]:
vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [16]:
for layer in vgg.layers:
  layer.trainable = False


In [17]:
x = Flatten()(vgg.output)

In [18]:
prediction = Dense(16,activation='softmax')(x)

In [19]:
model2 = Model(inputs=vgg.input,outputs=prediction)

In [20]:
model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
model2.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))

<ipython-input-21-93be31fd6a36>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(train,validation_data=test,epochs=10,steps_per_epoch=len(train),validation_steps=len(test))


Epoch 1/10
19/19 [==============================] - 96s 5s/step - loss: 3.1398 - accuracy: 0.3000 - val_loss: 3.7667 - val_accuracy: 0.2420
Epoch 2/10
19/19 [==============================] - 121s 7s/step - loss: 0.8910 - accuracy: 0.7267 - val_loss: 3.2368 - val_accuracy: 0.3503
Epoch 3/10
19/19 [==============================] - 89s 5s/step - loss: 0.3063 - accuracy: 0.9000 - val_loss: 3.5048 - val_accuracy: 0.2930
Epoch 4/10
19/19 [==============================] - 91s 5s/step - loss: 0.2223 - accuracy: 0.9467 - val_loss: 3.0112 - val_accuracy: 0.3822
Epoch 5/10
19/19 [==============================] - 98s 5s/step - loss: 0.0734 - accuracy: 0.9933 - val_loss: 3.1450 - val_accuracy: 0.4204
Epoch 6/10
19/19 [==============================] - 98s 5s/step - loss: 0.1074 - accuracy: 0.9733 - val_loss: 3.3157 - val_accuracy: 0.3822
Epoch 7/10
19/19 [==============================] - 97s 5s/step - loss: 0.0543 - accuracy: 0.9933 - val_loss: 3.0424 - val_accuracy: 0.4268
Epoch 8/10
19/19 [=

##Testing


In [22]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [23]:
output = ['blasti',
 'bonegl',
 'brhkyt',
 'cbrtsh',
 'cmnmyn',
 'gretit',
 'hilpig',
 'himbul',
 'himgri',
 'hsparo',
 'indvul',
 'jglowl',
 'lbicrw',
 'mgprob',
 'rebimg',
 'wcrsrt']

In [27]:
##Prediction using CNN Model
img = image.load_img("/content/drive/MyDrive/_D32_10313.jpg",target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred1 =  np.argmax(model.predict(img))
print(output[pred1])

1/1 [==============================] - 0s 126ms/step
blasti


In [28]:
##Prediction using CNN Model
img1 = image.load_img("/content/drive/MyDrive/_DSC9446.jpg",target_size=(224,224))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred1_1 =  np.argmax(model.predict(img1))
print(output[pred1_1])

1/1 [==============================] - 0s 20ms/step
blasti


In [29]:
##Prediction using VGG16 Model
img = image.load_img("/content/drive/MyDrive/100_4468.JPG",target_size=(224,224))
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
pred2 =  np.argmax(model2.predict(img))
print(output[pred2])

1/1 [==============================] - 2s 2s/step
wcrsrt
